In [1]:
import requests
import yaml
import json
import gzip
import pandas as pd

import os
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager

In [4]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [7]:
def make_Xy(convs, N=10):
    errors = ["Contradiction", "Self-contradiction"]
    
    X = []
    y = []
    for conv in convs:

        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_exist_error():
                X.append( dialogue[-N:] )
                    # X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
        
    return X, y

In [8]:
X_str, y = make_Xy(convs, N=5)

In [1]:
import copy
def predicate_argument_structure_analysis(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    suit_list = []
    for i, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
            # 受動表現の可能性があるものは回避
            if i<=len(doc)-2 and doc[i+1].pos_ == "AUX" and doc[i+1].lemma_ in ["れる", "られる"]:
                # print(doc)
                continue
            is_exist_case = False
            for c in token.children:
                if c.dep_ in ["nsubj", "obj", "obl"]:
                    noun = c.lemma_
                    for c2 in c.children:
                        if c2.dep_ == "case":
                            is_exist_case = True
                        # if c2.dep_ == "case":
                            case = c2.orth_
                            if case == "は":
                                case = "が"
                            triple_list.append( (noun, case, token.lemma_ ))
             
    return triple_list